In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:

In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

# Youtube Agent

In [ ]:
import requests

@tool
def search_youtube_video(query: str, region: str = "US", language: str = "en") -> dict:
    """
    Search for a video on YouTube using the RapidAPI YouTube138 API.

    Args:
        query (str): The search query (e.g., video title or keyword).
        region (str): The region code (default is 'US').
        language (str): The language code (default is 'en').

    Returns:
        dict: JSON containing details of the top video result.
    """
    url = "https://youtube138.p.rapidapi.com/search/"
    params = {
        "q": query,
        "hl": language,
        "gl": region
    }
    headers = {
        "x-rapidapi-key": "api key",  # Replace with your RapidAPI key
        "x-rapidapi-host": "youtube138.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        for result in data.get("contents", []):
            video = result.get("video")
            if video:
                video_id = video.get("videoId")
                return {
                    "query": query,
                    "video_id": video_id,
                    "video_url": f"https://www.youtube.com/watch?v={video_id}",
                    "title": video.get("title"),
                    "description": video.get("descriptionSnippet"),
                    "duration_seconds": video.get("lengthSeconds"),
                    "published_time": video.get("publishedTimeText"),
                    "views": video.get("stats", {}).get("views"),
                    "thumbnail": video.get("thumbnails", [{}])[0].get("url"),
                    "channel": {
                        "title": video.get("author", {}).get("title"),
                        "channel_id": video.get("author", {}).get("channelId"),
                        "avatar": video.get("author", {}).get("avatar", [{}])[0].get("url"),
                        "channel_url": f"https://www.youtube.com/channel/{video.get('author', {}).get('channelId')}"
                    }
                }

        return {"message": "No video found for your query."}

    except requests.exceptions.RequestException as e:
        return {"error": f"Error during YouTube search: {str(e)}"}
    except (KeyError, IndexError, TypeError) as e:
        return {"error": f"Unexpected response format: {str(e)}"}

In [ ]:
youtube_assistant = create_react_agent(
    model=model,
    tools=[search_youtube_video],
    prompt=(
        "You are a helpful YouTube video assistant. Your job is to help users find YouTube videos "
        "based on their search queries using the `search_youtube_video` tool.\n\n"
        "🧠 Think step-by-step and always call the tool using the user's query.\n"
        "📦 When returning the result to the user, **always format the output strictly as JSON** using the structure below:\n\n"
        "{\n"
        '  "query": string,\n'
        '  "video_id": string,\n'
        '  "video_url": string,\n'
        '  "title": string,\n'
        '  "description": string,\n'
        '  "duration_seconds": integer,\n'
        '  "published_time": string,\n'
        '  "views": integer,\n'
        '  "thumbnail": string,\n'
        '  "channel": {\n'
        '    "title": string,\n'
        '    "channel_id": string,\n'
        '    "avatar": string,\n'
        '    "channel_url": string\n'
        "  }\n"
        "}\n\n"
        "❗️Do not include any commentary or explanation outside the JSON. Only return valid JSON."
    ),
    name="youtube_assistant"
)


In [ ]:
response = youtube_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="Find a YouTube video that includes the following: - Types: meditation, breathing exercises, motivational - Keywords: stress management, mindfulness, sleep improvement - Mood: calm - Duration: medium (e.g., 0-10min=short, 10-30min=medium, 30min+=long)")
            ]
    }
    )
response["messages"]

[HumanMessage(content='Find a YouTube video that includes the following: - Types: meditation, breathing exercises, motivational - Keywords: stress management, mindfulness, sleep improvement - Mood: calm - Duration: medium (e.g., 0-10min=short, 10-30min=medium, 30min+=long)', additional_kwargs={}, response_metadata={}, id='7dc6daf6-df78-4fac-b207-b6398359a17c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YNU88aPYDDfpCa4VesNTLTv0', 'function': {'arguments': '{"query":"Types: meditation, breathing exercises, motivational; Keywords: stress management, mindfulness, sleep improvement; Mood: calm; Duration: medium"}', 'name': 'search_youtube_video'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 386, 'total_tokens': 427, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens

In [ ]:
response["messages"][-1].content

'{\n  "query": "Types: meditation, breathing exercises, motivational; Keywords: stress management, mindfulness, sleep improvement; Mood: calm; Duration: medium",\n  "video_id": "0E3cbfJnRBE",\n  "video_url": "https://www.youtube.com/watch?v=0E3cbfJnRBE",\n  "title": "Guided 4-7-8 Breathing Technique #relaxing #breathingexercise #meditation",\n  "description": "Enjoy this 4-7-8 breathing technique guided by visual and sounds. 4-7-8 breathing is a breathing technique that can help to ...",\n  "duration_seconds": 20,\n  "published_time": "8 months ago",\n  "views": 40695,\n  "thumbnail": "https://i.ytimg.com/vi/0E3cbfJnRBE/hq720_2.jpg?sqp=-oaymwE2COgCEMoBSFXyq4qpAygIARUAAIhCGABwAcABBvABAfgBtgiAAoAPigIMCAAQARg1IFQofzAP&rs=AOn4CLCNMKdPmZQcpz9pQBzumv81zUIdfA",\n  "channel": {\n    "title": "Headfulness - Luke Horton",\n    "channel_id": "UCAc9_Ug_adJvcKJhAIUqhIQ",\n    "avatar": "https://yt3.ggpht.com/K3EheypHS6P27o67_NVJArpZEDw9a-BK2j5LZ-ayVLHIGHU6sdoTPo2tNNEUHGaghDOjU4qmIck=s68-c-k-c0x00ff

# Spotify Agent

In [ ]:
from langchain.tools import tool
from typing import List
import requests

# Optional: Use a dataclass to format results more clearly
def get_spotify_token(client_id: str, client_secret: str) -> str:
    """
    Fetches a bearer token from Spotify using client credentials.
    """
    url = 'https://accounts.spotify.com/api/token'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'grant_type': 'client_credentials'}
    response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))
    response.raise_for_status()
    return response.json()['access_token']

import requests

@tool
def search_spotify_playlists(query: str) -> dict:
    """
    Search for Spotify playlists using a keyword (e.g., "sleep", "meditation").

    Returns a JSON object with the top 5 playlists including name, description, image, link, owner, and track count.
    """
    client_id = "api key"
    client_secret = "api key"

    try:
        token = get_spotify_token(client_id, client_secret)

        headers = {
            'Authorization': f'Bearer {token}'
        }
        params = {
            'q': query,
            'type': 'playlist',
            'limit': 5
        }
        url = 'https://api.spotify.com/v1/search'
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        playlists = response.json().get('playlists', {}).get('items', [])

        if not playlists:
            return {"message": "No playlists found for your query."}

        result = {
            "query": query,
            "total": len(playlists),
            "playlists": []
        }

        for pl in playlists:
            if pl is None:
                continue

            result["playlists"].append({
                "name": pl.get('name'),
                "description": pl.get('description'),
                "external_url": pl.get('external_urls', {}).get('spotify'),
                "image": pl.get('images', [{}])[0].get('url'),
                "owner": {
                    "name": pl.get('owner', {}).get('display_name'),
                    "url": pl.get('owner', {}).get('external_urls', {}).get('spotify')
                },
                "tracks": {
                    "url": pl.get('tracks', {}).get('href'),
                    "total": pl.get('tracks', {}).get('total')
                }
            })

        return result

    except Exception as e:
        return {"error": str(e)}

In [ ]:
spotify_assistant = create_react_agent(
    model=model,
    tools=[search_spotify_playlists],
    prompt=(
        "You are a helpful Spotify assistant. Your job is to help users discover Spotify playlists "
        "based on their search queries using the tool provided.\n\n"
        "🧠 Think step-by-step, and always call the `search_spotify_playlists` tool with the user query.\n"
        "📦 When returning results to the user, format the final output in **JSON** using this structure:\n\n"
        "{\n"
        '  "query": string,\n'
        '  "total": integer,\n'
        '  "playlists": [\n'
        "    {\n"
        '      "name": string,\n'
        '      "description": string,\n'
        '      "external_url": string,\n'
        '      "image": string,\n'
        '      "owner": {"name": string, "url": string},\n'
        '      "tracks": {"url": string, "total": integer}\n'
        "    },\n"
        "    ...\n"
        "  ]\n"
        "}\n\n"
        "Only return JSON. Do not add any extra commentary or text outside of the JSON structure."
    ),
    name="spotify_assistant"
)

# Sleep Agent Tools

In [ ]:
from langchain.tools import tool
from datetime import datetime, timedelta
import random


In [ ]:
@tool
def generate_wind_down_routine(sleep_quality: str) -> str:
    """
    Suggests a personalized wind-down routine based on sleep quality: 'good', 'average', or 'poor'.
    """
    routines = {
        'good': ['light reading', 'herbal tea', 'breathing exercises'],
        'average': ['light yoga', 'reading', 'screen off 1 hour before bed'],
        'poor': ['no screens 2 hrs before bed', 'guided meditation', 'warm bath', 'journaling']
    }
    routine = routines.get(sleep_quality.lower(), routines['average'])
    return f"Your wind-down routine should include: {', '.join(routine)}."


In [ ]:
@tool
def log_sleep(start_time: str, end_time: str) -> str:
    """
    Logs sleep session using start and end timestamps (in ISO format).
    """
    start = datetime.fromisoformat(start_time)
    end = datetime.fromisoformat(end_time)
    duration = (end - start).total_seconds() / 3600
    return f"Sleep logged from {start.strftime('%I:%M %p')} to {end.strftime('%I:%M %p')} ({duration:.2f} hours)."


In [ ]:
@tool
def get_sleep_reminders(bedtime: str) -> list:
    """
    Provides reminders for wind-down activities based on bedtime in HH:MM (24-hour format).
    """
    bed_hour = datetime.strptime(bedtime, "%H:%M")
    return [
        f"Avoid caffeine after {(bed_hour - timedelta(hours=6)).strftime('%I:%M %p')}.",
        f"Turn off screens by {(bed_hour - timedelta(hours=1)).strftime('%I:%M %p')}.",
        f"Begin wind-down routine by {(bed_hour - timedelta(minutes=30)).strftime('%I:%M %p')}."
    ]


In [ ]:
@tool
def generate_routine_from_history(avg_sleep_hours: float, interruptions: int) -> str:
    """
    Generates a new bedtime routine based on average sleep hours and interruptions.
    """
    if avg_sleep_hours < 6 or interruptions > 2:
        return "You should try avoiding screens 2 hours before bed, meditate 10 minutes, and dim lights early."
    elif 6 <= avg_sleep_hours < 7:
        return "Avoid screens 1 hour before sleep, drink herbal tea, and do light stretching."
    else:
        return "Maintain current routine but be consistent with sleep timing."


In [ ]:
@tool
def check_sleep_disorder(avg_sleep: float, latency: float, disturbances: int) -> str:
    """
    Suggests possible issues based on sleep parameters.
    - avg_sleep: average hours of sleep
    - latency: average time to fall asleep (in minutes)
    - disturbances: number of wake-ups per night
    """
    if avg_sleep < 5 and disturbances > 3:
        return "Risk of insomnia or sleep apnea detected. Consider consulting a doctor."
    elif latency > 45:
        return "You may be experiencing delayed sleep phase disorder."
    return "No significant disorder signs detected. Keep monitoring."


In [ ]:
@tool
def suggest_bedtime(wake_up_time: str, sleep_cycle_hours: float = 7.5) -> str:
    """
    Suggests an optimal bedtime given wake-up time and desired sleep cycle.
    """
    wake = datetime.strptime(wake_up_time, "%H:%M")
    bedtime = (wake - timedelta(hours=sleep_cycle_hours)).strftime('%I:%M %p')
    return f"You should aim to sleep by {bedtime}."


In [ ]:
@tool
def build_routine_with_calendar(next_day_meeting: str, sleep_deficit: float) -> str:
    """
    Builds next-day routine considering an early meeting and recent sleep deficit (in hours).
    """
    meeting_time = datetime.strptime(next_day_meeting, "%H:%M")
    if sleep_deficit > 2:
        return f"You're sleep-deprived. Wind down at {(meeting_time - timedelta(hours=9)).strftime('%I:%M %p')}."
    return f"Wind down at {(meeting_time - timedelta(hours=8)).strftime('%I:%M %p')}."


In [ ]:
@tool
def avoidance_timing(bedtime: str) -> str:
    """
    Provides guidance on when to avoid screen time, caffeine, and heavy exercise.
    """
    bed = datetime.strptime(bedtime, "%H:%M")
    screen_off = (bed - timedelta(hours=1)).strftime('%I:%M %p')
    last_caffeine = (bed - timedelta(hours=6)).strftime('%I:%M %p')
    no_exercise = (bed - timedelta(hours=2)).strftime('%I:%M %p')

    return (
        f"Avoid caffeine after {last_caffeine}. "
        f"Turn off screens by {screen_off}. "
        f"Avoid heavy exercise after {no_exercise}."
    )


In [ ]:
@tool
def sleep_checkin(feeling: str, woke_up_time: str) -> str:
    """
    Analyzes how the user felt upon waking up.
    """
    feedback = {
        'tired': "Consider going to bed earlier or improving wind-down habits.",
        'refreshed': "Great! Keep up the consistent routine.",
        'groggy': "Try cutting screen time and avoiding late meals."
    }
    return feedback.get(feeling.lower(), "Thanks for checking in. Monitor trends over a week for better insights.")


In [ ]:
@tool
def respond_to_query(query: str) -> str:
    """
    Responds to general user sleep-related queries.
    """
    if "how did I sleep" in query.lower():
        return "Your average sleep for the past week was 6.8 hours with 2 interruptions/night."
    elif "generate routine" in query.lower():
        return generate_wind_down_routine("poor")
    elif "remind me" in query.lower():
        return "Reminder set! You'll be notified 1 hour before bedtime."
    return "I'm here to help with sleep, routines, and reminders. Ask me anything!"


# Sleep Agent

In [ ]:

model = ChatOpenAI(api_key=key,model="gpt-4o-mini")
# List all your @tool functions here
tools = [
    generate_wind_down_routine,
    log_sleep,
    get_sleep_reminders,
    generate_routine_from_history,
    check_sleep_disorder,
    suggest_bedtime,
    build_routine_with_calendar,
    avoidance_timing,
    sleep_checkin,
    respond_to_query
]

sleep_coach_agent = create_react_agent(
    model=model,  # Replace with your preferred language model instance
    tools=tools,

    prompt=(
        "You are a smart and caring Sleep Coach Agent. Your job is to help users improve their sleep quality "
        "by analyzing their sleep data, generating personalized routines, setting reminders, and detecting any issues "
        "based on sleep patterns.\n\n"
        "🧠 You must ALWAYS reason step-by-step and choose the correct tool to help the user based on their input.\n\n"

        "Here’s how to respond:\n"
        "1. Identify the user’s intent.\n"
        "2. Use the correct tool based on the intent.\n"
        "3. Call the tool with appropriate arguments using the user’s input.\n"
        "4. Return a clean, helpful response directly to the user (unless the tool already formats it).\n\n"

        "📦 Here are the tools you have access to:\n"
        "- `generate_wind_down_routine`: Suggests bedtime routines based on sleep quality.\n"
        "- `log_sleep`: Logs the user's sleep session.\n"
        "- `get_sleep_reminders`: Generates reminders like when to stop screen use and begin wind-down.\n"
        "- `generate_routine_from_history`: Uses sleep hours and interruptions to build a new routine.\n"
        "- `check_sleep_disorder`: Predicts possible sleep-related health issues.\n"
        "- `suggest_bedtime`: Calculates optimal sleep time based on desired wake-up.\n"
        "- `build_routine_with_calendar`: Suggests sleep timing based on calendar and sleep debt.\n"
        "- `avoidance_timing`: Suggests when to avoid caffeine, screen, and exercise.\n"
        "- `sleep_checkin`: Lets users log how they feel after waking up and provides tips.\n"
        "- `respond_to_query`: General sleep assistant to answer open-ended questions.\n\n"

        "❗️Important:\n"
        "- Only use tools when needed.\n"
        "- Never fabricate information. Always reason using data or tool outputs.\n"
        "- Respond in a friendly, helpful tone.\n"
        "- When returning final answers, DO NOT include any internal thoughts or steps.\n"
    ),
    name="sleep_coach_agent",
)


In [ ]:
from langchain.schema import HumanMessage

response = sleep_coach_agent.invoke({
    "messages": [
        HumanMessage(content=(
            "I had poor sleep last night. Can you give me a good wind-down routine for tonight?"
            "I slept from 11 PM to 6:30 AM last night. Please log it."
            "Set reminders for when I should stop screen use and when to start winding down if I want to sleep by 10 PM."
        ))
    ]
})

response["messages"]


[HumanMessage(content='I had poor sleep last night. Can you give me a good wind-down routine for tonight?I slept from 11 PM to 6:30 AM last night. Please log it.Set reminders for when I should stop screen use and when to start winding down if I want to sleep by 10 PM.', additional_kwargs={}, response_metadata={}, id='e55b7221-4a6a-454b-8a9c-02bfba6ba39a'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Wl19deKYJMDauoVuWsh9RzCf', 'function': {'arguments': '{"sleep_quality": "poor"}', 'name': 'generate_wind_down_routine'}, 'type': 'function'}, {'id': 'call_Oa9KPYGfJUmHwBhlxfKPEAgq', 'function': {'arguments': '{"start_time": "2023-11-02T23:00:00", "end_time": "2023-11-03T06:30:00"}', 'name': 'log_sleep'}, 'type': 'function'}, {'id': 'call_gKdKeQzdL2eSZkEj27njRzuy', 'function': {'arguments': '{"bedtime": "22:00"}', 'name': 'get_sleep_reminders'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 827, 

In [ ]:
response["messages"][-1].content

"Here's your personalized wind-down routine for tonight:\n\n1. **Avoid screens** at least 2 hours before bed (so, no screens after 9:00 PM).\n2. **Start your wind-down routine** by 9:30 PM.\n3. Consider incorporating activities like:\n   - Guided meditation or relaxation exercises.\n   - A warm bath to relax your muscles.\n   - Journaling to clear your mind before sleep.\n\nYour sleep has been logged, and I’ve noted that you slept from 11:00 PM to 6:30 AM (totaling 7.5 hours).\n\n### Sleep Reminders for Tonight:\n- **Avoid caffeine** after 4:00 PM.\n- **Turn off screens** by 9:00 PM.\n- **Begin your wind-down routine** by 9:30 PM.\n\nThese steps should help you improve your sleep quality tonight! If you need further assistance, just let me know."

# Sleeep

In [ ]:
import json
import re
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate

class SleepCoachAgent:
    def __init__(self, api_key):
        self.llm = OpenAI(openai_api_key=api_key, model="gpt-4o-mini", max_tokens=4096)
        self.prompt = PromptTemplate(
    template="""
You are a smart and caring Sleep Coach AI. Analyze the user's sleep-related input and provide a structured response in JSON format.

CONTEXT:
{context}

ANALYSIS GOALS:
1. Identify user's sleep issue or goal
2. Provide a summary of their sleep situation
3. Generate a personalized wind-down routine
4. Suggest optimal sleep and wake times
5. Recommend actions for better sleep
6. Check for any potential sleep disorders
7. Provide a structured sleep plan

USER INPUT: "{input}"

RESPONSE FORMAT (valid JSON object only):
Double quotes must be used, and no comments or markdown formatting.

{{{{
  "sleepAssessment": {{{{
    "issue": "string",
    "confidence": number,
    "severity": "low" | "medium" | "high",
    "summary": "string",
    "sleepHistoryDetected": boolean
  }}}},
  "routineRecommendation": {{{{
    "windDown": ["string"],
    "avoidBeforeBed": ["string"],
    "optimalSleepTime": "string",
    "optimalWakeTime": "string",
    "reminders": ["string"]
  }}}},
  "tips": {{{{
    "immediateActions": ["string"],
    "lifestyleChanges": ["string"],
    "environmentSuggestions": ["string"]
  }}}},
  "disorderCheck": {{{{
    "riskLevel": "low" | "medium" | "high",
    "symptoms": ["string"],
    "recommendations": ["string"]
  }}}},
  "followUp": {{{{
    "checkInPeriod": "string",
    "trackMetrics": ["string"],
    "goals": ["string"]
  }}}},
  "metadata": {{{{
    "wordCount": number,
    "timeOfDayMentioned": "morning" | "afternoon" | "evening" | "night" | "unknown",
    "contextTags": ["string"]
  }}}}
}}}}

GUIDELINES:
- Do not include markdown formatting (like ```json)
- Do not add any text outside the JSON
- End your response with the closing }}
""",
    input_variables=["context", "input"]
)


#         self.prompt = PromptTemplate(
#             template="""
# You are a smart and caring Sleep Coach AI. Analyze the user's sleep-related input and provide a structured response in JSON format.

# `CONTEXT:
# {context}

# ANALYSIS GOALS:
# 1. Identify user's sleep issue or goal
# 2. Provide a summary of their sleep situation
# 3. Generate a personalized wind-down routine
# 4. Suggest optimal sleep and wake times
# 5. Recommend actions for better sleep
# 6. Check for any potential sleep disorders
# 7. Provide a structured sleep plan

# USER INPUT: "{input}"

# RESPONSE FORMAT (FULL JSON OBJECT! Starts with `{{`):
# ```json
# {{
#   "sleepAssessment": {{
#     "issue": "string", // e.g., can't fall asleep, frequent waking, not feeling rested
#     "confidence": number, // 0–100 confidence
#     "severity": "low | medium | high", // issue impact level
#     "summary": "string", // summary of what user is facing
#     "sleepHistoryDetected": boolean
#   }},
#   "routineRecommendation": {{
#     "windDown": ["string"], // steps like reading, shower, meditation
#     "avoidBeforeBed": ["string"], // caffeine, screens, etc.
#     "optimalSleepTime": "string", // e.g., 10:30 PM
#     "optimalWakeTime": "string", // e.g., 6:30 AM
#     "reminders": ["string"] // helpful reminders like 'no screens 1 hour before bed'
#   }},
#   "tips": {{
#     "immediateActions": ["string"], // actions they can take today
#     "lifestyleChanges": ["string"], // long-term sleep hygiene practices
#     "environmentSuggestions": ["string"] // room, lighting, temperature, etc.
#   }},
#   "disorderCheck": {{
#     "riskLevel": "low | medium | high",
#     "symptoms": ["string"], // list of warning signs
#     "recommendations": ["string"] // if high risk, suggest seeing a sleep specialist
#   }},
#   "followUp": {{
#     "checkInPeriod": "string", // e.g., in 3 days
#     "trackMetrics": ["string"], // what they should track (sleep time, restfulness, etc.)
#     "goals": ["string"] // e.g., 7.5 hrs sleep, consistent sleep schedule
#   }},
#   "metadata": {{
#     "wordCount": number,
#     "timeOfDayMentioned": "morning | afternoon | evening | night | unknown",
#     "contextTags": ["string"] // detected themes like stress, work, screen time, etc.
#   }}
# }}

# GUIDELINES:
# - Be warm, non-judgmental, and science-based
# - Suggest healthy sleep habits but avoid medical advice
# - If risk is high, suggest professional consultation
# - Tailor wind-down suggestions to stress levels, timing, and emotional state
# - JSON MUST be complete with all fields
# - END your response with a full JSON object. DO NOT truncate.
# `,
#   """,
#             input_variables=["context", "input"]
#         )

        self.chain = self.prompt | self.llm
        self.collection = self._get_collection()

    def _get_collection(self):
        # You can customize this to use a DB or tracking log if needed
        return []

    def run(self, user_input, context=None):
      try:
          context_str = """
          - MindFuel's Sleep Coach helps users improve sleep hygiene and build better bedtime routines.
          - Users may be experiencing insomnia, irregular schedules, screen overuse, or stress.
          - The agent provides actionable suggestions and helps track progress over time.
          - All suggestions should be safe, natural, and non-prescriptive.
          """

          prompt_vars = {
              "context": context_str,
              "input": user_input
          }

          result = self.chain.invoke(prompt_vars)

          print("-----RAW LLM OUTPUT-----")
          print(result)
          print("------------------------")

          try:
              # Try direct load first
              try:
                  return json.loads(result.strip())
              except json.JSONDecodeError:
                  # Wrap if the model returns a partial JSON (starting directly with keys)
                  fixed = result.strip()
                  if not fixed.strip().startswith("{"):
                      fixed = "{\n" + fixed + "\n}"
                  return json.loads(fixed)

          except Exception as parse_err:
              print("❌ JSON Parsing Error:", parse_err)
              return {"error": "Failed to parse JSON response."}

      except Exception as e:
          print("❌ Agent Run Error:", e)
          return {"error": str(e)}

    # def run(self, user_input, context=None):
    #     try:
    #         context_str = """
    #         - MindFuel's Sleep Coach helps users improve sleep hygiene and build better bedtime routines.
    #         - Users may be experiencing insomnia, irregular schedules, screen overuse, or stress.
    #         - The agent provides actionable suggestions and helps track progress over time.
    #         - All suggestions should be safe, natural, and non-prescriptive.
    #         """

    #         prompt_vars = {
    #             "context": context_str,
    #             "input": user_input
    #         }

    #         result = self.chain.invoke(prompt_vars)

    #         print("-----RAW LLM OUTPUT-----")
    #         print(result)
    #         print("------------------------")

    #         try:
    #             # Try direct JSON parse first
    #             try:
    #                 parsed_result = json.loads(result.strip())
    #             except json.JSONDecodeError:
    #                 # Fallback: regex extract
    #                 json_str = re.search(r"\{[\s\S]*\}", result).group(0)
    #                 parsed_result = json.loads(json_str)

    #             return parsed_result

    #         except Exception as parse_err:
    #             print("❌ JSON Parsing Error:", parse_err)
    #             return {"error": "Failed to parse JSON response."}

    #     except Exception as e:
    #         print("❌ Agent Run Error:", e)
    #         return {"error": str(e)}


In [ ]:
agent = SleepCoachAgent(api_key=key)
user_input = "I've been waking up tired even after 8 hours of sleep. I use my phone late at night."
response = agent.run(user_input)

import pprint
pprint.pprint(response)

-----RAW LLM OUTPUT-----
  
{
  "sleepAssessment": {
    "issue": "Waking up tired after 8 hours of sleep",
    "confidence": 0.85,
    "severity": "medium",
    "summary": "User reports feeling tired upon waking despite getting adequate sleep duration, potentially due to poor sleep quality or habits.",
    "sleepHistoryDetected": false
  },
  "routineRecommendation": {
    "windDown": [
      "Dim the lights in your room 30 minutes before bed.",
      "Engage in a relaxing activity such as reading a book or practicing gentle yoga.",
      "Use calming essential oils like lavender in your bedtime routine."
    ],
    "avoidBeforeBed": [
      "Avoid using your phone or other screens at least 1 hour before bed.",
      "Limit caffeine intake in the late afternoon and evening."
    ],
    "optimalSleepTime": "10:30 PM",
    "optimalWakeTime": "6:30 AM",
    "reminders": [
      "Set a bedtime alarm to remind you to wind down.",
      "Keep a consistent sleep schedule, even on weekends."


In [ ]:
agent = SleepCoachAgent(api_key=key)
response = agent.run("I sleep 8 hours but still feel tired in the morning. I often use my phone before bed.")
print(json.dumps(response, indent=2))


-----RAW LLM OUTPUT-----
  
{
  "sleepAssessment": {
    "issue": "Feeling tired after 8 hours of sleep",
    "confidence": 85,
    "severity": "medium",
    "summary": "User sleeps for 8 hours but still feels tired in the morning, possibly due to phone usage before bed.",
    "sleepHistoryDetected": false
  },
  "routineRecommendation": {
    "windDown": ["Read a book for 30 minutes", "Practice deep breathing exercises", "Listen to calming music"],
    "avoidBeforeBed": ["Using phone or other screens", "Caffeinated beverages", "Heavy meals"],
    "optimalSleepTime": "10:30 PM",
    "optimalWakeTime": "6:30 AM",
    "reminders": ["Set a bedtime alarm to remind you to start winding down", "Keep your phone in another room during bedtime"]
  },
  "tips": {
    "immediateActions": ["Limit screen time to 30 minutes before bed", "Create a relaxing bedtime ritual"],
    "lifestyleChanges": ["Consider reducing caffeine intake in the afternoon", "Incorporate regular physical activity during the

# Sleep 1

In [ ]:
import json
import re
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate

class SleepCoachAgent:
    def __init__(self, api_key):
        self.llm = OpenAI(openai_api_key=api_key, model="gpt-4o-mini", max_tokens=4096)
        self.prompt = PromptTemplate(
    template="""
You are a smart and caring Sleep Coach AI. Analyze the user's sleep-related input and provide a structured response in JSON format.

CONTEXT:
{context}

ANALYSIS GOALS:
1. Identify user's sleep issue or goal
2. Provide a summary of their sleep situation
3. Generate a personalized wind-down routine
4. Suggest optimal sleep and wake times
5. Recommend actions for better sleep
6. Check for any potential sleep disorders
7. Provide a structured sleep plan

USER INPUT: "{input}"

RESPONSE FORMAT (valid JSON object only):
Double quotes must be used, and no comments or markdown formatting.

{{{{
  "sleepAssessment": {{{{
    "issue": "string",
    "confidence": number,
    "severity": "low" | "medium" | "high",
    "summary": "string",
    "sleepHistoryDetected": boolean
  }}}},
  "routineRecommendation": {{{{
    "windDown": ["string"],
    "avoidBeforeBed": ["string"],
    "optimalSleepTime": "string",
    "optimalWakeTime": "string",
    "reminders": ["string"]
  }}}},
  "tips": {{{{
    "immediateActions": ["string"],
    "lifestyleChanges": ["string"],
    "environmentSuggestions": ["string"]
  }}}},
  "disorderCheck": {{{{
    "riskLevel": "low" | "medium" | "high",
    "symptoms": ["string"],
    "recommendations": ["string"]
  }}}},
  "followUp": {{{{
    "checkInPeriod": "string",
    "trackMetrics": ["string"],
    "goals": ["string"]
  }}}},
  "metadata": {{{{
    "wordCount": number,
    "timeOfDayMentioned": "morning" | "afternoon" | "evening" | "night" | "unknown",
    "contextTags": ["string"]
  }}}}
}}}}

GUIDELINES:
- Do not include markdown formatting (like ```json)
- Do not add any text outside the JSON
- End your response with the closing }}
""",
    input_variables=["context", "input"]
)


#         self.prompt = PromptTemplate(
#             template="""
# You are a smart and caring Sleep Coach AI. Analyze the user's sleep-related input and provide a structured response in JSON format.

# `CONTEXT:
# {context}

# ANALYSIS GOALS:
# 1. Identify user's sleep issue or goal
# 2. Provide a summary of their sleep situation
# 3. Generate a personalized wind-down routine
# 4. Suggest optimal sleep and wake times
# 5. Recommend actions for better sleep
# 6. Check for any potential sleep disorders
# 7. Provide a structured sleep plan

# USER INPUT: "{input}"

# RESPONSE FORMAT (FULL JSON OBJECT! Starts with `{{`):
# ```json
# {{
#   "sleepAssessment": {{
#     "issue": "string", // e.g., can't fall asleep, frequent waking, not feeling rested
#     "confidence": number, // 0–100 confidence
#     "severity": "low | medium | high", // issue impact level
#     "summary": "string", // summary of what user is facing
#     "sleepHistoryDetected": boolean
#   }},
#   "routineRecommendation": {{
#     "windDown": ["string"], // steps like reading, shower, meditation
#     "avoidBeforeBed": ["string"], // caffeine, screens, etc.
#     "optimalSleepTime": "string", // e.g., 10:30 PM
#     "optimalWakeTime": "string", // e.g., 6:30 AM
#     "reminders": ["string"] // helpful reminders like 'no screens 1 hour before bed'
#   }},
#   "tips": {{
#     "immediateActions": ["string"], // actions they can take today
#     "lifestyleChanges": ["string"], // long-term sleep hygiene practices
#     "environmentSuggestions": ["string"] // room, lighting, temperature, etc.
#   }},
#   "disorderCheck": {{
#     "riskLevel": "low | medium | high",
#     "symptoms": ["string"], // list of warning signs
#     "recommendations": ["string"] // if high risk, suggest seeing a sleep specialist
#   }},
#   "followUp": {{
#     "checkInPeriod": "string", // e.g., in 3 days
#     "trackMetrics": ["string"], // what they should track (sleep time, restfulness, etc.)
#     "goals": ["string"] // e.g., 7.5 hrs sleep, consistent sleep schedule
#   }},
#   "metadata": {{
#     "wordCount": number,
#     "timeOfDayMentioned": "morning | afternoon | evening | night | unknown",
#     "contextTags": ["string"] // detected themes like stress, work, screen time, etc.
#   }}
# }}

# GUIDELINES:
# - Be warm, non-judgmental, and science-based
# - Suggest healthy sleep habits but avoid medical advice
# - If risk is high, suggest professional consultation
# - Tailor wind-down suggestions to stress levels, timing, and emotional state
# - JSON MUST be complete with all fields
# - END your response with a full JSON object. DO NOT truncate.
# `,
#   """,
#             input_variables=["context", "input"]
#         )

        self.chain = self.prompt | self.llm
        self.collection = self._get_collection()

    def _get_collection(self):
        # You can customize this to use a DB or tracking log if needed
        return []

    def run(self, user_input, context=None):
      try:
          context_str = """
          - MindFuel's Sleep Coach helps users improve sleep hygiene and build better bedtime routines.
          - Users may be experiencing insomnia, irregular schedules, screen overuse, or stress.
          - The agent provides actionable suggestions and helps track progress over time.
          - All suggestions should be safe, natural, and non-prescriptive.
          """

          prompt_vars = {
              "context": context_str,
              "input": user_input
          }

          result = self.chain.invoke(prompt_vars)

          print("-----RAW LLM OUTPUT-----")
          print(result)
          print("------------------------")

          try:
              # Try direct load first
              try:
                  return json.loads(result.strip())
              except json.JSONDecodeError:
                  # Wrap if the model returns a partial JSON (starting directly with keys)
                  fixed = result.strip()
                  if not fixed.strip().startswith("{"):
                      fixed = "{\n" + fixed + "\n}"
                  return json.loads(fixed)

          except Exception as parse_err:
              print("❌ JSON Parsing Error:", parse_err)
              return {"error": "Failed to parse JSON response."}

      except Exception as e:
          print("❌ Agent Run Error:", e)
          return {"error": str(e)}




In [ ]:
agent = SleepCoachAgent(api_key=key)
user_input = "I've been waking up tired even after 8 hours of sleep. I use my phone late at night."
response = agent.run(user_input)

import pprint
pprint.pprint(response)

-----RAW LLM OUTPUT-----
}} 

{
  "sleepAssessment": {
    "issue": "Waking up tired despite 8 hours of sleep",
    "confidence": 0.85,
    "severity": "medium",
    "summary": "User reports feeling tired in the morning, indicating possible disruptions in sleep quality, potentially due to late-night phone use.",
    "sleepHistoryDetected": false
  },
  "routineRecommendation": {
    "windDown": ["Turn off all screens at least 30 minutes before bed", "Read a book or listen to calming music", "Practice gentle stretching or relaxation exercises"],
    "avoidBeforeBed": ["Using your phone", "Caffeine consumption", "Heavy meals"],
    "optimalSleepTime": "10:30 PM",
    "optimalWakeTime": "6:30 AM",
    "reminders": ["Set a nightly alarm to remind you to start winding down"]
  },
  "tips": {
    "immediateActions": ["Reduce phone usage before bedtime", "Try a sleep meditation app"],
    "lifestyleChanges": ["Establish a consistent sleep schedule", "Limit caffeine intake in the afternoon"],


In [ ]:
agent = SleepCoachAgent(api_key=key)
response = agent.run("I sleep 8 hours but still feel tired in the morning. I often use my phone before bed.")
print(json.dumps(response, indent=2))


-----RAW LLM OUTPUT-----
 
{
  "sleepAssessment": {
    "issue": "Feeling tired despite adequate sleep",
    "confidence": 85,
    "severity": "medium",
    "summary": "User sleeps 8 hours but wakes up feeling tired, potentially linked to screen use before bed.",
    "sleepHistoryDetected": false
  },
  "routineRecommendation": {
    "windDown": [
      "Read a physical book or magazine for 30 minutes",
      "Practice gentle stretching or yoga",
      "Listen to calming music or nature sounds"
    ],
    "avoidBeforeBed": [
      "Using phone or other screens",
      "Caffeinated beverages",
      "Heavy meals"
    ],
    "optimalSleepTime": "10:30 PM",
    "optimalWakeTime": "6:30 AM",
    "reminders": [
      "Set a timer to start winding down 30 minutes before bedtime.",
      "Keep your phone out of the bedroom or use blue light filters."
    ]
  },
  "tips": {
    "immediateActions": [
      "Limit screen time to 30 minutes before bed.",
      "Establish a consistent bedtime rout

# Sleep 2 (Main)

In [ ]:
import json
import re
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate

class SleepCoachAgent:
    def __init__(self, api_key):
        self.llm = OpenAI(openai_api_key=api_key, model="gpt-4o-mini", max_tokens=4096)
        self.prompt = PromptTemplate(
    template="""
You are a smart and caring Sleep Coach AI. Analyze the user's sleep-related input and provide a structured response in JSON format.

CONTEXT:
{context}

ANALYSIS GOALS:
1. Identify user's sleep issue or goal
2. Provide a summary of their sleep situation
3. Generate a personalized wind-down routine
4. Suggest optimal sleep and wake times
5. Recommend actions for better sleep
6. Check for any potential sleep disorders
7. Provide a structured sleep plan

USER INPUT: "{input}"

RESPONSE FORMAT (valid JSON object only):
Double quotes must be used, and no comments or markdown formatting.

{{{{
  "sleepAssessment": {{{{
    "issue": "string",
    "confidence": number,
    "severity": "low" | "medium" | "high",
    "summary": "string",
    "sleepHistoryDetected": boolean
  }}}},
  "routineRecommendation": {{{{
    "windDown": ["string"],
    "avoidBeforeBed": ["string"],
    "optimalSleepTime": "string",
    "optimalWakeTime": "string",
    "reminders": ["string"]
  }}}},
  "tips": {{{{
    "immediateActions": ["string"],
    "lifestyleChanges": ["string"],
    "environmentSuggestions": ["string"]
  }}}},
  "disorderCheck": {{{{
    "riskLevel": "low" | "medium" | "high",
    "symptoms": ["string"],
    "recommendations": ["string"]
  }}}},
  "followUp": {{{{
    "checkInPeriod": "string",
    "trackMetrics": ["string"],
    "goals": ["string"]
  }}}},
  "metadata": {{{{
    "wordCount": number,
    "timeOfDayMentioned": "morning" | "afternoon" | "evening" | "night" | "unknown",
    "contextTags": ["string"]
  }}}}
}}}}

GUIDELINES:
- Do not include markdown formatting (like ```json)
- Do not add any text outside the JSON
- End your response with the closing }}
""",
    input_variables=["context", "input"]
)
        self.chain = self.prompt | self.llm
        self.collection = self._get_collection()

    def _get_collection(self):
        # You can customize this to use a DB or tracking log if needed
        return []

    def run(self, user_input, context=None):
      try:
          context_str = """
          - MindFuel's Sleep Coach helps users improve sleep hygiene and build better bedtime routines.
          - Users may be experiencing insomnia, irregular schedules, screen overuse, or stress.
          - The agent provides actionable suggestions and helps track progress over time.
          - All suggestions should be safe, natural, and non-prescriptive.
          """

          prompt_vars = {
              "context": context_str,
              "input": user_input
          }

          result = self.chain.invoke(prompt_vars)

          # print("-----RAW LLM OUTPUT-----")
          # print(result)
          # print("------------------------")

          try:
              # Try direct load first
              try:
                  return json.loads(result.strip())
              except json.JSONDecodeError:
                  # Wrap if the model returns a partial JSON (starting directly with keys)
                  fixed = result.strip()
                  if not fixed.strip().startswith("{"):
                      fixed = "{\n" + fixed + "\n}"
                  return json.loads(fixed)

          except Exception as parse_err:
              print("❌ JSON Parsing Error:", parse_err)
              return {"error": "Failed to parse JSON response."}

      except Exception as e:
          print("❌ Agent Run Error:", e)
          return {"error": str(e)}




In [ ]:
agent = SleepCoachAgent(api_key=key)
response = agent.run("I sleep 8 hours but still feel tired in the morning. I often use my phone before bed.")
print(json.dumps(response, indent=2))


{
  "sleepAssessment": {
    "issue": "Feeling tired after 8 hours of sleep",
    "confidence": 80,
    "severity": "medium",
    "summary": "User is getting sufficient sleep duration but feels fatigued in the morning, likely due to phone usage before bed.",
    "sleepHistoryDetected": false
  },
  "routineRecommendation": {
    "windDown": [
      "Read a physical book or listen to calming music",
      "Practice relaxation techniques such as deep breathing or meditation",
      "Dim the lights an hour before bed"
    ],
    "avoidBeforeBed": [
      "Using the phone or any screens",
      "Caffeine consumption in the evening",
      "Heavy meals close to bedtime"
    ],
    "optimalSleepTime": "10:30 PM",
    "optimalWakeTime": "6:30 AM",
    "reminders": [
      "Set an alarm for wind-down time",
      "Limit screen time to 30 minutes before bed"
    ]
  },
  "tips": {
    "immediateActions": [
      "Stop using the phone at least 30 minutes before bed",
      "Create a bedtime rout

# Youtube + Spotify

In [ ]:
import json

agent = MoodAnalyzerAgent()

# Provide sample input
user_input = "Lately, I've been feeling overwhelmed with work and unable to focus. I'm not sleeping well, and I get easily irritated."

# Run the agent
output = agent.run(user_input)

# Print the output nicely
print(json.dumps(output, indent=2))

-----RAW LLM OUTPUT-----



USER INPUT: "Lately, I've been feeling overwhelmed with work and unable to focus. I'm not sleeping well, and I get easily irritated."
            """

            {
                "analysis": {
                    "primaryMood": "Frustrated",
                    "moodCategory": "negative",
                    "confidence": 90,
                    "intensity": 8,
                    "emotions": [
                        {
                            "emotion": "Overwhelmed",
                            "score": 85
                        },
                        {
                            "emotion": "Irritated",
                            "score": 75
                        },
                        {
                            "emotion": "Anxious",
                            "score": 70
                        },
                        {
                            "emotion": "Tired",
                            "score": 80
                       